# <span style="color:#0b486b">SIT307 - Data Mining and Machine Learning</span>

---
Lecturer:   Richard Dazeley     | richard.dazeley@deakin.edu.au<br />
Assistant:  Adam Bignold | abignold@gmail.com

School of Information Technology, <br />
Deakin University, VIC 3216, Australia.


---


## <span style="color:#0b486b">Practical Session 2: Data and Visualisations with pandas</span>

**Prerequisite**
You should already have done, or be confident with the content of: 
1. Week 1 material

**The purpose of this session is:**

1. learn simple data and file operations using `pandas`
2. plot graphs using `Matplotlib` with `pandas`

**Instructions** 

1. After you download this notebook, save it as another copy and rename it to `"[yourstudentID]_Week_2_Data_and_Visualisations_with_Pandas.ipynb"`
2. fill in the code cells indicated with your own solution. You can discuss approaches with other students but must only submit your own original solution. 

## <span style="color:#0b486b">Pandas: Quick Introduction </span>
Pandas is a Python package providing fast, flexible, and expressive data structures designed to make working with “relational” or “labeled” data both easy and intuitive.

Pandas is well suited for many different kinds of data:
* Tabular data with heterogeneously-typed columns, as in an SQL table or Excel spreadsheet
* Ordered and unordered (not necessarily fixed-frequency) time series data.
* Arbitrary matrix data (homogeneously typed or heterogeneous) with row and column labels
* Any other form of observational / statistical data sets. The data actually need not be labeled at all to be placed into a pandas data structure

For more information see http://pandas.pydata.org/pandas-docs/stable/. 

### <span style="color:#0b486b">Reading in Data </span>
Let’s start with reading a small comma-separated (CSV) text file – ex1.csv, since this is comma-delimited, we can use read_csv to read it into a DataFrame (assumes you have down loaded the zip file and it is in the working): 

In [ ]:
import pandas as pd
df = pd.read_csv('data\\ex1.csv')
df

In [ ]:
pd.read_table('data\\ex1.csv', sep=',')

A file will not always have a header row – ex2.csv. To read this file, you have a couple of options. You can allow pandas to assign default column names by setting `header = None:` 

In [ ]:
pd.read_csv('data\\ex2.csv', header = None)

Or you can provide your own names by setting names to an array of names. Try setting the columns to the following five `names = ['Gogo Yubari', 'Louis Gara', 'The Gimp', 'Captain Koons', 'Vernita Green']`

In [ ]:
pd.read_csv('data\\ex2.csv', names=['Gogo Yubari', 'Louis Gara', 'The Gimp', 'Captain Koons', 'Vernita Green'])

In some cases, a table might not have a fixed delimiter, using whitespace or some other pattern to separate fields (see ex3.csv with notepad++). While you could do some munging by hand, the fields here are separated by a variable amount of whitespace. In these cases, you can pass a regular expression as a delimiter for read_table. This can be expressed by the regular expression \s+, so we have then. 

In [ ]:
pd.read_table('data\\ex3.txt', sep ='\s+', names=['Pai Mei', 'Mallory Knox', 'O-Ren Ishii', 'Marsellus Wallace'])

The parser functions have many additional arguments to help you handle the wide variety of exception file formats that occur. For example, you can skip the first, third, and fourth rows of a file with skiprows in ex4.csv

In [ ]:
pd.read_csv('data\\ex4.csv')

In [ ]:
pd.read_csv('data\\ex4.csv', skiprows=[0, 2, 3])

Handling missing values is an important and frequently nuanced part of the file parsing process. Missing data is usually either not present (empty string) or marked by some sentinel value. By default, pandas uses a set of commonly occurring sentinels, such as NA and NULL (see ex5.csv):

In [ ]:
result = pd.read_csv('data\\ex5.csv')
pd.isnull(result)

### <span style="color:#0b486b">Reading files in pieces </span>
When processing very large files or figuring out the right set of arguments to correctly process a large file, you may only want to read in a small piece of a file or iterate through smaller chunks of the file. Before we look at a large file, we make the pandas display settings more compact (ex6.csv). 

In [ ]:
pd.read_csv('data\\ex6.csv')

If you want to only read a small number of rows (avoiding reading the entire file), specify that with nrows. 

In [ ]:
pd.read_csv('data\\ex6.csv', nrows=5)

To read a file in pieces, specify a chunksize as a number of rows. 

In [ ]:
pd.read_csv('data\\ex6.csv', chunksize=1000)

This returns a TextParser object which allows you to iterate over the parts of the file according to the chunksize. For example, we can iterate over ex6.csv, aggregating the value counts in the 'key' column like so:

In [ ]:
chunker = pd.read_csv('data\\ex6.csv', chunksize=5)
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)
tot[:100]

Writing Data to Text Format Data can also be exported to a delimited format. Let’s consider one of the CSV files read before (ex5.csv), using DataFrame’s to_csv method, we can write the data out to a comma-separated file (out.csv):

In [ ]:
data=pd.read_csv('data\\ex5.csv')
data.to_csv('data\\out.csv')

### <span style="color:#0b486b">JSON Data </span>
JSON (short for JavaScript Object Notation) has become one of the standard formats for sending data by HTTP request between web browsers and other applications. It is a much more free-form data format than a tabular text form like CSV.  JSON is very nearly valid Python code with the exception of its null value null and some other nuances (such as disallowing trailing commas at the end of lists). The basic types are objects (dicts), arrays (lists), strings, numbers, booleans, and nulls. All of the keys in an object must be strings. There are several Python libraries for reading and writing JSON data. Module json will be used here, as it is built into the Python standard library. To convert a JSON string to Python form, use json.loads: 
 
 

In [ ]:
import json
obj = """
{"name": "Wes", 
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 30, "pets":["Zeus", "Zuko"]},
{"name": "Katie", "age":38,
"pets": ["Sixes", "Stache", "Cisco"]}]
}
"""
result = json.loads(obj)
result

The pandas.read_json can automatically convert JSON datasets in specific arrangements into a Series or DataFrame. The default options for pandas.read_json assume that each object in the JSON array is a row in the table: 

In [ ]:
data = pd.read_json('data\\example.json')
data

### <span style="color:#0b486b">Binary Data Formats </span>
One of the easiest ways to store data (also known as serialization) efficiently in binary format is using Python’s built-in pickle serialization. pandas objects all have a to_pickle method that writes the data to disk in pickle format. You can read any “pickled” object stored in a file by using the built-in pickle directly, or even more conveniently using pandas.read_pickle:

In [ ]:
frame = pd.read_csv('data\\ex1.csv')
frame.to_pickle('data\\frame_pickle')
pd.read_pickle('data\\frame_pickle')

### <span style="color:#0b486b">HDF5 Format </span>
HDF5 is a well-regarded file format intended for storing large quantities of scientific array data. It is available as a C library, and it has interfaces available in many other languages, including Java, Julia, MATLAB, and Python. The “HDF” in HDF5 stands for hierarchical data format. Each HDF5 file can store multiple datasets and supporting metadata. Compared with simpler formats, HDF5 supports on-the-fly compression with a variety of compression modes, enabling data with repeated patterns to be stored more efficiently. HDF5 can be a good choice for working with very large datasets that don’t fit into memory, as you can efficiently read and write small sections of much larger arrays. While it’s possible to directly access HDF5 files using either the PyTables or h5py libraries, pandas provides a high-level interface that simplifies storing Series and DataFrame object. The HDFStore class works like a dict and handles the low-level details: 

In [ ]:
import pandas as pd
import numpy as np

frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('data\\mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

In [ ]:
store['obj1']

### <span style="color:#0b486b">Reading Microsoft Excel Files </span>
pandas also supports reading tabular data stored in Excel 2003 (and higher) files using either the ExcelFile class or pandas.read_excel function. Internally these tools use the add-on packages xlrd and openpyxl to read XLS and XLSX files, respectively. You may need to install these manually with pip or conda. To use ExcelFile, create an instance by passing a path to an xls or xlsx file: 
 
 

In [ ]:
import pandas as pd
import numpy as np

xlsx = pd.ExcelFile('data\\ex1.xlsx')
pd.read_excel(xlsx, 'Sheet1')

If you are reading multiple sheets in a file, then it is faster to create the ExcelFile, but you can also simply pass the filename to pandas.read_excel: 

In [ ]:
frame = pd.read_excel('data\\ex1.xlsx', 'Sheet1')
frame

To write pandas data to Excel format, you must first create an ExcelWriter, then write data to it using pandas objects’ to_excel method: 

In [ ]:
import pandas as pd
import numpy as np

frame = pd.read_excel('data\\ex1.xlsx', 'Sheet1')
writer = pd.ExcelWriter('data\\ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

### <span style="color:#0b486b">Interacting with Web APIs </span>
Many websites have public APIs providing data feeds via JSON or some other format. There are a number of ways to access these APIs from Python; one easy-to-use method is the requests package. To find the last 30 GitHub issues for pandas on GitHub, we can make a GET HTTP request using the add-on requests library: 

In [ ]:
import pandas as pd
import requests

url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
data = resp.json()
data[0]['title']

In [ ]:
issues = pd.DataFrame(data,columns=['number', 'title', 'labels', 'state'])
issues

### <span style="color:#0b486b">Interacting with Databases  </span>
In a business setting, most data may not be stored in text or Excel files. SQL-based relational databases (such as SQL Server, PostgreSQL, and MySQL) are in wide use, and many alternative databases have become quite popular. The choice of database is usually dependent on the performance, data integrity, and scalability needs of an application. Loading data from SQL into a DataFrame is straightforward, and pandas has some functions to simplify the process. As an example, a SQLite database is created using Python’s built-in sqlite3 driver: 

In [ ]:
import pandas as pd
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
c REAL, d INTEGER
); """

In [ ]:
con = sqlite3.connect('data\\mydata.sqlite')
con.execute(query)
con.commit()
con

In [ ]:
#insert 3 rows of data
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

In [ ]:
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [ ]:
#select data from a table:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

## <span style="color:#0b486b">2. Matplotlib</span>

In this section we will quickly look at using Matplotlib with pandas
The plot method on `Series` and `DataFrame` is just a simple wrapper around `plt.plot():`

Section based on https://pandas.pydata.org/pandas-docs/stable/visualization.html.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))
ts = ts.cumsum()
ts.plot()

By giving the index dates, it calls `gcf().autofmt_xdate()` to format the x-axis.

On `DataFrame`, `plot()` is a convenience to plot all of the columns with labels:

In [ ]:
df = pd.DataFrame(np.random.randn(1000, 4), index=ts.index, columns=list('ABCD'))
df = df.cumsum()
plt.figure()
df.plot()

You can also plot one column against a second using the x and y keywords in plot():

In [ ]:
df3 = pd.DataFrame(np.random.randn(1000, 2), columns=['B', 'C']).cumsum()
df3['A'] = pd.Series(list(range(len(df))))
df3.plot(x='A', y='B')

### <span style="color:#0b486b">Bar plot</span>


In [ ]:
plt.figure();

df.iloc[5].plot(kind='bar');

In [ ]:
df2 = pd.DataFrame(np.random.rand(10, 4), columns=['a', 'b', 'c', 'd'])

df2.plot.bar();

In [ ]:
df2.plot.bar(stacked=True);

In [ ]:
df2.plot.barh(stacked=True);

### <span style="color:#0b486b">Histograms</span>
Histograms can be drawn by using the `DataFrame.plot.hist()` and `Series.plot.hist()` methods.

In [ ]:
df4 = pd.DataFrame({'a': np.random.randn(1000) + 1, 'b': np.random.randn(1000),
                    'c': np.random.randn(1000) - 1}, columns=['a', 'b', 'c'])
plt.figure();
df4.plot.hist(alpha=0.5)

In [ ]:
df4.plot.hist(stacked=True, bins=20)

### <span style="color:#0b486b">Box Plot</span>
Histograms can be drawn by using the DataFrame.plot.hist() and Series.plot.hist() methods.

In [ ]:
df = pd.DataFrame(np.random.rand(10, 5), columns=['A', 'B', 'C', 'D', 'E'])

df.plot.box()

Boxplot can be colorized by passing `color` keyword. You can pass a `dict` whose keys are `boxes`, `whiskers`, `medians` and `caps`. If some keys are missing in the `dict`, default colors are used for the corresponding artists. Also, boxplot has `sym` keyword to specify fliers style.

When you pass other type of arguments via `color` keyword, it will be directly passed to `matplotlib` for all the `boxes`, `whiskers`, `medians` and `caps` colorization.

The colors are applied to every boxes to be drawn. If you want more complicated colorization, you can get each drawn artists by passing return_type.

In [ ]:
color = dict(boxes='DarkGreen', whiskers='DarkOrange',
             medians='DarkBlue', caps='Gray')


df.plot.box(color=color, sym='r+')

In [ ]:
df.plot.box(vert=False, positions=[1, 4, 5, 6, 8])

### <span style="color:#0b486b">Area Plot</span>
You can create area plots with `Series.plot.area()` and `DataFrame.plot.area()`. Area plots are stacked by default. To produce stacked area plot, each column must be either all positive or all negative values.

When input data contains `NaN`, it will be automatically filled by 0. If you want to drop or fill by different values, use `dataframe.dropna()` or `dataframe.fillna()` before calling plot.

In [ ]:
df = pd.DataFrame(np.random.rand(10, 4), columns=['a', 'b', 'c', 'd'])

df.plot.area();

To produce an unstacked plot, pass `stacked=False`. Alpha value is set to `0.5` unless otherwise specified:

In [ ]:
df.plot.area(stacked=False);

### <span style="color:#0b486b">Scatter Plot</span>
Scatter plot can be drawn by using the `DataFrame.plot.scatter()` method. Scatter plot requires numeric columns for the `x` and `y` axes. These can be specified by the `x` and `y` keywords.

In [ ]:
df = pd.DataFrame(np.random.rand(50, 4), columns=['a', 'b', 'c', 'd'])
df.plot.scatter(x='a', y='b');

To plot multiple column groups in a single axes, repeat `plot` method specifying target `ax`. It is recommended to specify `color` and `label` keywords to distinguish each groups.

In [ ]:
ax = df.plot.scatter(x='a', y='b', color='DarkBlue', label='Group 1');
df.plot.scatter(x='c', y='d', color='DarkGreen', label='Group 2', ax=ax);

The keyword `c` may be given as the name of a column to provide colors for each point:

In [ ]:
df.plot.scatter(x='a', y='b', c='c', s=50);

You can pass other keywords supported by matplotlib `scatter`. The example below shows a bubble chart using a column of the `DataFrame` as the bubble size.

In [ ]:
df.plot.scatter(x='a', y='b', s=df['c']*200);

There are significantly more options to plot graphs from pandas. See the pandas [cookbook](https://pandas.pydata.org/pandas-docs/stable/cookbook.html#cookbook-plotting) for more options.